In [1]:
import json
import tweepy
from tweepy import OAuthHandler, Stream
from tweepy.streaming import StreamListener
from kafka import KafkaProducer, KafkaConsumer
from datetime import datetime
import time

In [2]:
topic = 'spotify'
word_filter = ['spotify com track']
# language_filter = ['en']

access_token = ''       #your access token
access_secret = ''      #your access token secret
consumer_key = ''       #your consumer key 
consumer_secret = ''    #your consumer key secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [5]:
class KafkaPushListener(StreamListener):
    def __init__(self, time_limit = 3600):
        print("Publish data to topic: "+topic)
        self.producer = KafkaProducer(bootstrap_servers='localhost:9092')
        self.start_time = time.time()
        self.limit = time_limit
        
    def normalize_timestamp(self, time):
        mytime = datetime.strptime(time, "%a %b %d %H:%M:%S +0000 %Y")
        return(mytime.strftime("%Y-%m-%d %H:%M:%S"))
    
    def on_data(self, data):
        json_object = json.loads(data)
        if (time.time() - self.start_time) < self.limit:
            if len(json_object['entities']['urls']) > 0:
                record = ''
                record += str(self.normalize_timestamp(str(json_object['created_at'])))
                record += '|'
                record += str(json_object['user']['screen_name'].encode('utf-8'))
                record += '|'
                record += str(json_object['entities']['urls'][0]['expanded_url'])
                self.producer.send(topic, str.encode(record))
                print(record)
            return True
        else:
            self.producer.close()
            return False

In [6]:
if __name__ == '__main__':
    twitter_stream = Stream(auth, KafkaPushListener())
    twitter_stream.filter(track=word_filter)

Publish data to topic: testing
2020-09-25 03:55:29|b'phantasmofist'|https://open.spotify.com/track/1Cwsd5xI8CajJz795oy4XF?si=HuYQjjv3Rz26vKZoKEQOlQ
2020-09-25 03:55:29|b'chenvisual'|https://twitter.com/i/web/status/1309340722617868289
2020-09-25 03:55:29|b'filppa_petri'|https://open.spotify.com/track/1gmXQGru8mdJ1x19RzoUvA?si=1m9FUdBBRYqLGAJ9QplMHg
2020-09-25 03:55:29|b'RebeccaCristi20'|https://twitter.com/i/web/status/1309340723070803968
2020-09-25 03:55:29|b'DylanCrozier_'|https://open.spotify.com/track/3XV5r1cMB2AlD2ir0v6mfm?si=aRZCx8bfROa0WWkKODwVJQ
2020-09-25 03:55:29|b'raimlchr'|https://open.spotify.com/track/1dLXLETYiZohIZWnse4ypG?si=pPPY7TybTFSnPEcyk8j3Yg
2020-09-25 03:55:29|b'directionbabys2'|https://open.spotify.com/album/7MkNlHw0O8KKr3yJ3RtgVW?si=REEos9fuR02MupS8CfD_DQ
2020-09-25 03:55:30|b'chenleyes'|https://open.spotify.com/track/2sOz5KStj3M1L1ErNo6MCC?si=yfld2bDzQh-g6NYuITF1dg
2020-09-25 03:55:30|b'ClipsCray'|https://open.spotify.com/track/7sa8AlYpSZDDFQpQCDXq82?si=_evbm7